Homework 4: Neural Language Models (& 🎃 SpOoKy 👻 authors 🧟 data) - Task 2
----

### Names
----
Names: __William Aoun__ (Write these in every notebook you submit.)

Task 2: Training your own word embeddings (15 points)
--------------------------------

For this task, you'll use the `gensim` package to train your own embeddings for both words and characters. These will eventually act as inputs to your neural language model.

In [1]:
# here are several dependencies to install
!python --version
!python -m pip install --upgrade pip setuptools wheel
!pip install nltk
!pip install gensim
!pip install torch torchvision torchinfo

Python 3.11.9
  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


In [3]:
# import your libraries here

# Remember to restart your kernel if you change the contents of this file!
import neurallm_utils_starter as nutils

# for word embeddings
# if not installed, run the following command:
# !pip install gensim
from gensim.models import Word2Vec

import torch
import torch.nn as nn

[nltk_data] Downloading package punkt to /Users/billyaoun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/billyaoun/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
# If running on google colab, you'll need to mount your drive to access data files

#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
# constants you may find helpful. Edit as you would like.

# The dimensions of word embedding. 
# This variable will be used throughout the program
# DO NOT WRITE "50" WHEN YOU ARE REFERRING TO THE EMBEDDING SIZE
EMBEDDINGS_SIZE = 50

EMBEDDING_SAVE_FILE_WORD = f"spooky_embedding_word_{EMBEDDINGS_SIZE}.model" # The file to save your word embeddings to
EMBEDDING_SAVE_FILE_CHAR = f"spooky_embedding_char_{EMBEDDINGS_SIZE}.model" # The file to save your char embeddings to
TRAIN_FILE = 'spooky_author_train.csv' # The file to train your language model on
NGRAM = 3 

Train embeddings on provided dataset
---

In [ ]:
# your code here
# use the provided utility functions to read in the data


data = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
			['this', 'is', 'the', 'second', 'sentence'],
			['yet', 'another', 'sentence'],
			['one', 'more', 'sentence'],
			['and', 'the', 'final', 'sentence']]


# read the spooky data in both by character and by word using the read_file_spooky function in the
# provided utils
word_data = nutils.read_file_spooky(TRAIN_FILE, ngram=NGRAM, by_character=False)
char_data = nutils.read_file_spooky(TRAIN_FILE, ngram=NGRAM, by_character=True)

# print out the first two sentences in each format
# make sure we can read the output easily without scrolling to the side too much

print("word format:")
for i in range(2):
    print(f"Sentence {i+1}: {word_data[i]}")

print("\nchar format:")
for i in range(2):
    tokens = char_data[i]
    chunks = [tokens[j : j + 20] for j in range(0, len(tokens), 20)]
    print(f"sentence {i+1}:")
    for chunk in chunks:
        print(f"  {chunk}")

word format:
Sentence 1: ['<s>', '<s>', 'this', 'process', ',', 'however', ',', 'afforded', 'me', 'no', 'means', 'of', 'ascertaining', 'the', 'dimensions', 'of', 'my', 'dungeon', ';', 'as', 'i', 'might', 'make', 'its', 'circuit', ',', 'and', 'return', 'to', 'the', 'point', 'whence', 'i', 'set', 'out', ',', 'without', 'being', 'aware', 'of', 'the', 'fact', ';', 'so', 'perfectly', 'uniform', 'seemed', 'the', 'wall', '.', '</s>', '</s>']
Sentence 2: ['<s>', '<s>', 'it', 'never', 'once', 'occurred', 'to', 'me', 'that', 'the', 'fumbling', 'might', 'be', 'a', 'mere', 'mistake', '.', '</s>', '</s>']

char format:
sentence 1:
  ['<s>', '<s>', 't', 'h', 'i', 's', '_', 'p', 'r', 'o', 'c', 'e', 's', 's', ',', '_', 'h', 'o', 'w', 'e']
  ['v', 'e', 'r', ',', '_', 'a', 'f', 'f', 'o', 'r', 'd', 'e', 'd', '_', 'm', 'e', '_', 'n', 'o', '_']
  ['m', 'e', 'a', 'n', 's', '_', 'o', 'f', '_', 'a', 's', 'c', 'e', 'r', 't', 'a', 'i', 'n', 'i', 'n']
  ['g', '_', 't', 'h', 'e', '_', 'd', 'i', 'm', 'e', 'n', 's'

8. What character represents spaces when we tokenize by character? __underscore__
9. Read the word2vec documentation. What do the following parameters signify?
    - embeddings_size: __The dimensionality of the dense vector representation for each word__
    - window: __The max distance between the current and predicted word within a sentence__
    - min_count: __Ignores all words with total frequency lower that this value__
    - sg: __Represents skip-gram when sg=1 and CBOW when sg=0__

In [10]:
# 10 points
# create your word embeddings
# use the skip gram algorithm and a window size of 5
# min_count should be 1
# takes ~3.3 sec on Felix's computer for character embeddings using skip-gram with window size 5
# takes ~3.3 sec on Felix's computer for word embeddings using skip-gram with window size 5 


def train_word2vec(data: list[list[str]], embeddings_size: int,
                    window: int = 5, min_count: int = 1, sg: int = 1) -> Word2Vec:
    """
    Create new word embeddings based on our data.

    Params:
        data: The corpus
        embeddings_size: The dimensions in each embedding

    Returns:
        A gensim Word2Vec model
        https://radimrehurek.com/gensim/models/word2vec.html
    """
    model = Word2Vec(data, vector_size=embeddings_size, window=window, 
                    min_count=min_count, sg=sg)
    return model


# After you are happy with this function, copy + paste it into the bottom of 
# your neurallm_utils.py file
# You'll need it for the next task!
def create_embedder(raw_embeddings: Word2Vec) -> torch.nn.Embedding:
    """
    Create a PyTorch embedding layer based on our data.

    We will *first* train a Word2Vec model on our data.
    Then, we'll use these weights to create a PyTorch embedding layer.
        `nn.Embedding.from_pretrained(weights)`


    PyTorch docs: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding.from_pretrained
    Gensim Word2Vec docs: https://radimrehurek.com/gensim/models/word2vec.html

    Pay particular attention to the *types* of the weights and the types required by PyTorch.

    Params:
        data: The corpus
        embeddings_size: The dimensions in each embedding

    Returns:
        A PyTorch embedding layer
    """

    # Hint:
    # For later tasks, we'll need two mappings: One from token to index, and one from index to tokens.
    # It might be a good idea to store these as properties of your embedder.
    # e.g. `embedder.token_to_index = ...`
    
    weights = torch.FloatTensor(raw_embeddings.wv.vectors)
   
    embedder = torch.nn.Embedding.from_pretrained(weights)
    
    embedder.token_to_index = {token: idx for idx, token in enumerate(raw_embeddings.wv.index_to_key)}
    embedder.index_to_token = {idx: token for token, idx in embedder.token_to_index.items()}
    
    return embedder

In [11]:
# Create and save both sets (word and character based) of Word2Vec embeddings.
# Use the provided utility functions in nutils.
# These will be (re)loaded in the next notebook.
word_data = nutils.read_file_spooky(TRAIN_FILE, ngram=1, by_character=False)
char_data = nutils.read_file_spooky(TRAIN_FILE, ngram=1, by_character=True)

word_embeddings = train_word2vec(word_data, EMBEDDINGS_SIZE)
char_embeddings = train_word2vec(char_data, EMBEDDINGS_SIZE)

nutils.save_word2vec(word_embeddings, EMBEDDING_SAVE_FILE_WORD)
nutils.save_word2vec(char_embeddings, EMBEDDING_SAVE_FILE_CHAR)

In [12]:
# load them in again to make sure that this works and is still fast
word_embeddings = nutils.load_word2vec(EMBEDDING_SAVE_FILE_WORD)
char_embeddings = nutils.load_word2vec(EMBEDDING_SAVE_FILE_CHAR)

In [13]:
# now create the embedders
word_embedder = create_embedder(word_embeddings)
char_embedder = create_embedder(char_embeddings)

In [16]:
# take a look at your saved token to index and index to token mappings in your embedders to make sure they make sense
# AND that they are both dictionaries mapping from int to str or vice versa!
# don't leave a ton of output in your notebook when you turn it in, but you need to understand this,
# and it's an easy place to make a mistake that's hard to debug later.
# do leave whatever code you use here, comment it out if it produces a lot of output

print("Word embedder mappings:")
print(f"token_to_index type: {type(word_embedder.token_to_index)}")
print(f"index_to_token type: {type(word_embedder.index_to_token)}")
print(f"token_to_index: {list(word_embedder.token_to_index.items())[:5]}")
print(f"index_to_token: {list(word_embedder.index_to_token.items())[:5]}")

print("\nChar embedder mappings:")
print(f"token_to_index type: {type(char_embedder.token_to_index)}")
print(f"index_to_token type: {type(char_embedder.index_to_token)}")
print(f"token_to_index: {list(char_embedder.token_to_index.items())[:5]}")
print(f"index_to_token: {list(char_embedder.index_to_token.items())[:5]}")

Word embedder mappings:
token_to_index type: <class 'dict'>
index_to_token type: <class 'dict'>
token_to_index: [(',', 0), ('the', 1), ('of', 2), ('<s>', 3), ('</s>', 4)]
index_to_token: [(0, ','), (1, 'the'), (2, 'of'), (3, '<s>'), (4, '</s>')]

Char embedder mappings:
token_to_index type: <class 'dict'>
index_to_token type: <class 'dict'>
token_to_index: [('_', 0), ('e', 1), ('t', 2), ('a', 3), ('o', 4)]
index_to_token: [(0, '_'), (1, 'e'), (2, 't'), (3, 'a'), (4, 'o')]


In [17]:
# 4 points
# print out the vocabulary size for your embeddings for both your word
# embeddings and your character embeddings
# label which is which when you print them out
print(f"Word embeddings vocab size: {len(word_embedder.token_to_index)}")
print(f"Char embeddings vocab size: {len(char_embedder.token_to_index)}")

Word embeddings vocab size: 25374
Char embeddings vocab size: 60
